In [1]:
import xarray as xr

%run -i wave_stats.py
%run -i parse_spc_file.py

data_dir = 'F:/crs/proj/2025_NOPP_comparison/helene_adcirc_model_results/spec_files/'
h = 30.
normx, normy = 0.7071, 0.7071
ds_spec = read_swan_spec(data_dir+"bnd300.spc", h, normx, normy )
ds_spec

parse_spc_file.py loaded.


<xarray.Dataset> Size: 6MB
Dimensions:        (time: 145, freq: 41, dir: 36)
Coordinates:
  * time           (time) datetime64[ns] 1kB 2024-09-25 ... 2024-09-28
  * freq           (freq) float64 328B 0.0314 0.0345 0.038 ... 1.174 1.291 1.42
  * dir            (dir) float64 288B 5.0 15.0 25.0 35.0 ... 335.0 345.0 355.0
Data variables: (12/24)
    S2d            (time, freq, dir) float32 856kB 0.0 0.0 0.0 ... 0.0 0.0 0.0
    Ef             (time, freq) float32 24kB 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    Dth            (time, dir) float32 21kB 9.365e-10 1.038e-08 ... 4.04e-05
    m0             (time) float32 580B 0.000288 0.0002747 ... 0.005966 0.005702
    Hm0            (time) float32 580B 0.06788 0.06629 0.06505 ... 0.309 0.3021
    m1             (time) float64 1kB 0.0001027 9.814e-05 ... 0.0009186
    ...             ...
    Dir_flux_R     (time) float32 580B 0.7142 0.7251 0.7376 ... 0.6884 0.6868
    EfluxS_2d      (time, freq, dir) float64 2MB 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0
    EfluxS_2d_pos  (time, freq, dir) float64 2MB 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0
    EfluxS_f       (time, freq) float64 48kB 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    EfluxS_dir     (time, dir) float64 42kB 9.715e-05 0.0009226 ... 12.45 11.91
    EfluxS_total   (time) float64 1kB 0.1364 0.1514 0.1654 ... 30.28 28.57 26.92
Attributes:
    source:     SWAN .spec reader (QUANT once, repeated date-and-time blocks;...
    units_S2d:  m2/Hz/degr
    quantity:   VaDens